In [ ]:
import pickle

In [ ]:
def score_reduce_func(D_chunk, start, top_size=500):
    # Находим индексы ближайших соседей
    nearest_items = np.argsort(D_chunk, axis=1)[:, :top_size + 1]
    
    # Извлекаем соответствующие косинусные расстояния
    nearest_distances = np.take_along_axis(D_chunk, nearest_items, axis=1)
    
    # Возвращаем пары (индексы и близости)
    return [
        (i, items[items != i], 1 - nearest_distances[idx][items != i])  # 1 - расстояние для получения близости
        for idx, (i, items) in enumerate(zip(range(start, start + D_chunk.shape[0]), nearest_items))
    ]

In [ ]:
predictions_for_stack = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=score_reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items, query_similarities in chunk_result:
        # Добавляем результаты: (ID трека запроса, [ID соседей], [косинусные близости соседей])
        predictions_for_stack.append(
            (trackids[query_indx],
             [trackids[nn_indx] for nn_indx in query_nearest_items],
             query_similarities.tolist())  # Преобразуем близости в список
        )

In [ ]:
result_dict = {}

# Проходимся по списку predictions и заполняем словарь
for track_id, candidate_ids, similarities in predictions_for_stack:
    # Создаем пары (candidate_id, similarity) и добавляем в словарь
    result_dict[track_id] = list(zip(candidate_ids, similarities))

In [ ]:
def create_result_dict_with_model_scores(embeds_df):
    embeddings = embeds_df['embedding'].tolist()
    trackids = embeds_df.index.tolist()
    
    predictions_for_stack = []
    for chunk_result in tqdm(pairwise_distances_chunked(
            embeddings, metric='cosine', reduce_func=score_reduce_func, working_memory=100
    )):
        for query_indx, query_nearest_items, query_similarities in chunk_result:
            # Добавляем результаты: (ID трека запроса, [ID соседей], [косинусные близости соседей])
            predictions_for_stack.append(
                (trackids[query_indx],
                 [trackids[nn_indx] for nn_indx in query_nearest_items],
                 query_similarities.tolist())  # Преобразуем близости в список
            )
            
    result_dict = {}
    # Проходимся по списку predictions и заполняем словарь
    for track_id, candidate_ids, similarities in predictions_for_stack:
        # Создаем пары (candidate_id, similarity) и добавляем в словарь
        result_dict[track_id] = list(zip(candidate_ids, similarities))
    
    return result_dict

In [ ]:
result_dict = create_result_dict_with_model_scores(emb_pd)